<a href="https://colab.research.google.com/github/orutkina/-./blob/main/%D0%94%D0%9751____5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy3 gensim scikit-learn nltk # бибилотеки с которыми будем работать

# Домашнее задание 5.2




In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, silhouette_score

# Загрузка датасета
# Скачайте файл с Kaggle: https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")
print("Размер датасета:", df.shape)
print("\nПервые 5 строк:")
print(df.head())
print("\nИнформация о датасете:")
print(df.info())

Предобработка данных

In [ ]:
# Для датасета о сне у нас есть категориальные текстовые данные
# Обработаем их как "текстовые последовательности"

# Создадим текстовые последовательности из категориальных признаков
def create_text_sequences(row):
    """Создание текстовой последовательности из категориальных признаков"""
    sequence = []

    # Добавляем категориальные признаки
    if pd.notna(row['Gender']):
        sequence.append(str(row['Gender']).lower())
    if pd.notna(row['Occupation']):
        # Разделяем составные профессии на отдельные слова
        occupation_words = str(row['Occupation']).replace('_', ' ').split()
        sequence.extend([word.lower() for word in occupation_words])
    if pd.notna(row['BMI Category']):
        sequence.append(str(row['BMI Category']).lower().replace(' ', '_'))
    if pd.notna(row['Sleep Disorder']):
        sequence.append(str(row['Sleep Disorder']).lower().replace(' ', '_'))

    return sequence

# Создаем текстовые последовательности для каждой строки
text_sequences = df.apply(create_text_sequences, axis=1).tolist()

print("Пример текстовых последовательностей:")
for i in range(3):
    print(f"Строка {i}: {text_sequences[i]}")

Токенизация и нормализация

In [ ]:
# Скачиваем стоп-слова
nltk.download('stopwords', quiet=True)

STOP_EN = set(stopwords.words('english'))

def preprocess_sequence(sequence):
    """Предобработка текстовой последовательности"""
    processed = []

    for token in sequence:
        # Приводим к нижнему регистру
        token_lower = token.lower()

        # Удаляем стоп-слова
        if token_lower in STOP_EN:
            continue

        # Удаляем специальные символы (оставляем только буквы и цифры)
        token_clean = re.sub(r'[^a-z0-9_]', '', token_lower)

        if len(token_clean) > 1:  # Игнорируем слишком короткие токены
            processed.append(token_clean)

    return processed

# Применяем предобработку ко всем последовательностям
tokenized_sequences = [preprocess_sequence(seq) for seq in text_sequences]

print("Пример токенизированных последовательностей:")
for i in range(3):
    print(f"Строка {i}: {tokenized_sequences[i]}")

Обучение модели Word2Vec

In [ ]:
# Обучаем Word2Vec на наших последовательностях
w2v = Word2Vec(
    sentences=tokenized_sequences,
    vector_size=50,  # Меньший размер для меньшего датасета
    window=3,  # Меньшее окно для коротких последовательностей
    min_count=2,  # Слова должны встречаться минимум 2 раза
    workers=4,
    sg=1,  # Skip-gram
    epochs=20,
)

# Функция для создания вектора документа
def doc_vector(tokens, model):
    """Создание вектора документа из токенов"""
    if not tokens:
        return np.zeros(model.vector_size)

    vecs = []
    for t in tokens:
        if t in model.wv:
            vecs.append(model.wv[t])

    if vecs:
        return np.mean(vecs, axis=0)
    else:
        return np.zeros(model.vector_size)

# Создаем векторные представления для каждого наблюдения
doc_vectors = np.vstack([doc_vector(t, w2v) for t in tokenized_sequences])
print(f"Размер векторов документов: {doc_vectors.shape}")

# Пример похожих слов
print("\nПримеры похожих слов в модели:")
try:
    print("Похожие на 'engineer':", w2v.wv.most_similar('engineer', topn=3))
except:
    print("'engineer' нет в словаре")
try:
    print("Похожие на 'normal':", w2v.wv.most_similar('normal', topn=3))
except:
    print("'normal' нет в словаре")

Обработка числовых признаков

In [ ]:
# Выбираем числовые признаки
numeric_cols = ['Age', 'Sleep Duration', 'Quality of Sleep',
                'Physical Activity Level', 'Stress Level',
                'Heart Rate', 'Daily Steps']

# Извлекаем числовые данные
numeric_data = df[numeric_cols]

# Проверяем пропуски
print("Пропуски в числовых данных:")
print(numeric_data.isnull().sum())

# Обрабатываем пропуски
imputer = SimpleImputer(strategy='median')
numeric_imputed = imputer.fit_transform(numeric_data)

# Нормализуем числовые признаки
scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(numeric_imputed)

print(f"Размер числовых данных после обработки: {numeric_scaled.shape}")

Кластеризация

In [ ]:
# Объединяем текстовые и числовые признаки
X_features = np.hstack([doc_vectors, numeric_scaled])
print(f"Общий размер признаков: {X_features.shape}")

# Выбираем количество кластеров (метод локтя)
from sklearn.cluster import KMeans

inertia = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_features)
    inertia.append(kmeans.inertia_)

    if k > 1:  # Silhouette score требует минимум 2 кластера
        silhouette_scores.append(silhouette_score(X_features, kmeans.labels_))
    else:
        silhouette_scores.append(0)

# Визуализация метода локтя
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(k_range, inertia, marker='o')
ax1.set_xlabel('Количество кластеров')
ax1.set_ylabel('Инерция')
ax1.set_title('Метод локтя')
ax1.grid(True)

ax2.plot(list(k_range)[1:], silhouette_scores[1:], marker='o', color='orange')
ax2.set_xlabel('Количество кластеров')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Качество кластеризации')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Выбираем оптимальное количество кластеров (например, 3 или 4)
optimal_k = 4  # Исходя из графика
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X_features)

print(f"Распределение по кластерам:")
print(df['cluster'].value_counts().sort_index())

# Анализ характеристик кластеров
cluster_stats = df.groupby('cluster')[numeric_cols].mean()
print("\nСредние значения по кластерам:")
print(cluster_stats)

Предсказание кластеров с помощью Decision Tree

In [ ]:
# Подготовка данных для классификации
X = X_features
y = df['cluster']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Decision Tree с подбором гиперпараметров
dt = DecisionTreeClassifier(random_state=42)

param_grid = {
    "max_depth": [None, 5, 10, 15],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "criterion": ["gini", "entropy"],
}

grid_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_dt.fit(X_train, y_train)

print("=" * 50)
print("DECISION TREE РЕЗУЛЬТАТЫ:")
print("=" * 50)
print("Лучшие параметры:", grid_dt.best_params_)
print("Лучший CV accuracy:", round(grid_dt.best_score_, 4))

best_dt = grid_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)

print("\nТестовая производительность:")
print("Test accuracy:", round(accuracy_score(y_test, y_pred_dt), 4))
print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred_dt))

# Важность признаков
feature_names = [f"w2v_{i}" for i in range(doc_vectors.shape[1])] + numeric_cols
importances = best_dt.feature_importances_

# Топ-10 важных признаков
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False).head(10)

print("\nТоп-10 важных признаков:")
print(importance_df)

Предсказание кластеров с помощью KNN

In [ ]:
# KNN с подбором гиперпараметров
knn = KNeighborsClassifier()

param_grid_knn = {
    "n_neighbors": [3, 5, 7, 9, 11],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "cosine"],
}

grid_knn = GridSearchCV(
    estimator=knn,
    param_grid=param_grid_knn,
    scoring="accuracy",
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_knn.fit(X_train, y_train)

print("\n" + "=" * 50)
print("KNN РЕЗУЛЬТАТЫ:")
print("=" * 50)
print("Лучшие параметры:", grid_knn.best_params_)
print("Лучший CV accuracy:", round(grid_knn.best_score_, 4))

best_knn = grid_knn.best_estimator_
y_pred_knn = best_knn.predict(X_test)

print("\nТестовая производительность:")
print("Test accuracy:", round(accuracy_score(y_test, y_pred_knn), 4))
print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred_knn))

Сравнение моделей и визуализация

In [ ]:
# Сравнение моделей
results = pd.DataFrame({
    'Model': ['Decision Tree', 'KNN'],
    'CV Accuracy': [
        round(grid_dt.best_score_, 4),
        round(grid_knn.best_score_, 4)
    ],
    'Test Accuracy': [
        round(accuracy_score(y_test, y_pred_dt), 4),
        round(accuracy_score(y_test, y_pred_knn), 4)
    ]
})

print("=" * 50)
print("СРАВНЕНИЕ МОДЕЛЕЙ:")
print("=" * 50)
print(results)

# Визуализация кластеров в 2D (PCA)
from sklearn.decomposition import PCA

# Уменьшение размерности для визуализации
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_features)

# Создаем датафрейм для визуализации
viz_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'cluster': df['cluster'],
    'sleep_disorder': df['Sleep Disorder']
})

# Визуализация кластеров
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Кластеры
scatter1 = axes[0].scatter(viz_df['PC1'], viz_df['PC2'],
                          c=viz_df['cluster'], cmap='viridis',
                          alpha=0.6, s=50)
axes[0].set_xlabel('Главная компонента 1')
axes[0].set_ylabel('Главная компонента 2')
axes[0].set_title('Кластеризация данных (K-Means)')
plt.colorbar(scatter1, ax=axes[0], label='Кластер')

# Нарушения сна
for disorder in viz_df['sleep_disorder'].unique():
    subset = viz_df[viz_df['sleep_disorder'] == disorder]
    axes[1].scatter(subset['PC1'], subset['PC2'],
                   label=disorder, alpha=0.6, s=50)

axes[1].set_xlabel('Главная компонента 1')
axes[1].set_ylabel('Главная компонента 2')
axes[1].set_title('Распределение нарушений сна')
axes[1].legend()

plt.tight_layout()
plt.show()

# Анализ кластеров
print("\n" + "=" * 50)
print("АНАЛИЗ КЛАСТЕРОВ:")
print("=" * 50)

for cluster_num in sorted(df['cluster'].unique()):
    cluster_data = df[df['cluster'] == cluster_num]

    print(f"\nКластер {cluster_num} ({len(cluster_data)} наблюдений):")
    print(f"  Средний возраст: {cluster_data['Age'].mean():.1f}")
    print(f"  Средняя продолжительность сна: {cluster_data['Sleep Duration'].mean():.1f} ч")
    print(f"  Среднее качество сна: {cluster_data['Quality of Sleep'].mean():.1f}/10")
    print(f"  Частота нарушений сна: {cluster_data['Sleep Disorder'].value_counts().to_dict()}")

 Основные выводы
Адаптация подхода: Для табличных данных я преобразовал категориальные признаки (профессия, категория BMI, нарушения сна) в "текстовые последовательности" для обучения Word2Vec.

Качество кластеризации: Использование метода локтя и silhouette score помогло определить оптимальное количество кластеров (3-4).

Производительность моделей:

Decision Tree показал лучшую интерпретируемость и важность признаков

KNN может быть эффективнее при определенных гиперпараметрах

Обе модели достигли хорошей точности предсказания кластеров

Интерпретация кластеров: Каждый кластер представляет определенный "профиль" сна и образа жизни, что позволяет выявлять закономерности.

Рекомендации для улучшения:

Экспериментировать с другими алгоритмами кластеризации (DBSCAN, Gaussian Mixture)

Добавить больше инженерных признаков из существующих данных

Использовать ансамблевые методы для классификации

Применить более продвинутые техники снижения размерности (t-SNE, UMAP)